# Modeling

## 필요 라이브러리 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.4 MB/s 
     |████████████████████████████████| 182 kB 98.7 MB/s 
     |████████████████████████████████| 7.6 MB 87.0 MB/s 


## KoGPT2 model, tokenizer 호출

In [ ]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel
import torch


model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


## GPU 사용 여부

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model.to(device)
model.eval()
print(device)

cuda:0


## 파인 튜닝 데이터

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 발라드 약 56,000곡의 가사 데이터
f = open("/content/drive/MyDrive/Colab Notebooks/data/ballad_all.txt", 'r')
lines = f.readlines()
f.close()

In [ ]:
# 문장 개수, 약 96만개
print(len(lines))

961858


## tokenizer

In [ ]:
# # 문장 500,000개만 샘플링해서 일단 돌려보자
# man_lines = list(np.random.choice(lines, 500000))
# removed_lines = [line.replace('\n', '') for line in man_lines]
# list(np.random.choice(removed_lines, 10))

In [ ]:
removed_lines = [line.replace('\n', '') for line in lines[600000:]]
removed_lines[:5]

['수줍게 내게 다가와',
 '외로운 새벽에 익숙해질까',
 '작은 니 손을 잡고 있는 나',
 '1분 1초 매 순간 기억해',
 '눈물 마저 지쳐 그댈 잊기전에']

In [ ]:
tokenized_datasets = tokenizer(removed_lines, 
                               return_tensors="pt", 
                               padding="max_length", 
                               max_length=42,
                               truncation=True)

In [ ]:
tokenized_datasets

{'input_ids': tensor([[ 9025,  8241,  6866,  ...,     3,     3,     3],
        [ 9256,  9520, 46107,  ...,     3,     3,     3],
        [ 9836, 11523, 15309,  ...,     3,     3,     3],
        ...,
        [39194, 17077,  6889,  ...,     3,     3,     3],
        [10351, 25867, 12487,  ...,     3,     3,     3],
        [12817,  8168, 13278,  ...,     3,     3,     3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
# https://huggingface.co/transformers/custom_datasets.html
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    """ CustomDataset class for poetic sentences """
    def __init__(self, list_dataset, tokenizer):

        self.list_dataset = list_dataset
        self.tokenizer = tokenizer
        self.tokenized_sentences = self.tokenizer(
            list_dataset,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=42,
            add_special_tokens=True,
            return_token_type_ids=False,
            )

    def __getitem__(self, idx):
        encoded_dict = {key: val[idx] for key, val in self.tokenized_sentences.items()}
        encoded_dict["labels"] = encoded_dict["input_ids"].clone() # gpt has same labels as input_ids: https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb
        return encoded_dict

    def __len__(self):
        return len(self.list_dataset)

In [ ]:
tokenized_datasets = CustomDataset(removed_lines, tokenizer)

## 파인 튜닝

In [ ]:
from transformers import TrainingArguments

# Trainer가 학습, 평가에 사용할 모든 하이퍼 파라미터를 포함하는 클래스 정의
# 학습된 모델이 저장될 디렉토리만 지정하고 나머지는 기본값 사용
training_args = TrainingArguments(
    # 모델 저장 경로
    "/content/drive/MyDrive/Colab Notebooks/ballad_all_model",
    # 에포크 수
    num_train_epochs=10,
    # 메모리 절약
    gradient_checkpointing=False,
    # 매번 로그 찍을 스텝
    logging_steps=10000,
    warmup_steps=100000,
    save_steps=100000,
    eval_steps=100000)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 361858
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 452330
  Number of trainable parameters = 125164032


Step,Training Loss
10000,0.619300
20000,0.576500
30000,0.574900
40000,0.571100
50000,0.558400
60000,0.551100
70000,0.556600
80000,0.559500
90000,0.567300
100000,0.523500


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-100000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-100000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-100000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-100000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-100000/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-200000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-200000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoint-200000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/ballad_all_model/checkpoin

TrainOutput(global_step=452330, training_loss=0.43137149279062714, metrics={'train_runtime': 26928.626, 'train_samples_per_second': 134.377, 'train_steps_per_second': 16.797, 'total_flos': 7.756104900096e+16, 'train_loss': 0.43137149279062714, 'epoch': 10.0})

## 모델 저장

In [ ]:
repo_name = "wumusill/final_project_kogpt2"
token = "huggingface_token"

In [ ]:
## Upload to Huggingface Hub
model.push_to_hub(
    repo_name, 
    use_temp_dir=True, 
    use_auth_token=token
)
tokenizer.push_to_hub(
    repo_name, 
    use_temp_dir=True, 
    use_auth_token=token
)

Configuration saved in /tmp/tmpxy6nzxxw/config.json
Model weights saved in /tmp/tmpxy6nzxxw/pytorch_model.bin
Uploading the following files to wumusill/final_project_kogpt2: config.json,pytorch_model.bin
tokenizer config file saved in /tmp/tmpq6dwlga6/tokenizer_config.json
Special tokens file saved in /tmp/tmpq6dwlga6/special_tokens_map.json
Uploading the following files to wumusill/final_project_kogpt2: special_tokens_map.json,tokenizer.json,tokenizer_config.json


CommitInfo(commit_url='https://huggingface.co/wumusill/final_project_kogpt2/commit/3b601d060d227925046f35a500df23a6370cfad3', commit_message='Upload tokenizer', commit_description='', oid='3b601d060d227925046f35a500df23a6370cfad3', pr_url=None, pr_revision=None, pr_num=None)

## 모델 호출

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("wumusill/final_project_kogpt2")
model = AutoModelForCausalLM.from_pretrained("wumusill/final_project_kogpt2")

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

## 삼행시 함수 1

In [ ]:
def long_line_poem(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]


        while True:
            # 만약 idx 가 0 이라면 == 첫 글자
            if idx == 0:
                # 첫 글자 인코딩
                input_ids = tokenizer.encode(
                val, add_special_tokens=False, return_tensors="pt")
                # print(f"{idx}번 인코딩 : {input_ids}\n") # 2차원 텐서

                # 첫 글자 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=input_ids.to(device), 
                    do_sample=True, max_length=42,
                    min_length=5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)[0]
                # print("첫 글자 인코딩 후 generate 결과:", output_sequence, "\n") # tensor

            # 첫 글자가 아니라면
            else:
                # 한 음절
                input_ids = tokenizer.encode(
                val, add_special_tokens=False, return_tensors="pt")
                # print(f"{idx}번 째 글자 인코딩 : {input_ids} \n")

                # 좀더 매끄러운 삼행시를 위해 이전 인코딩과 지금 인코딩 연결
                link_with_pre_sentence = torch.cat((generated_sequence, input_ids[0]), 0)
                link_with_pre_sentence = torch.reshape(link_with_pre_sentence, (1, len(link_with_pre_sentence)))
                # print(f"이전 텐서와 연결된 텐서 {link_with_pre_sentence} \n")

                # 인코딩 값으로 문장 생성
                output_sequence = model.generate(
                    input_ids=link_with_pre_sentence.to(device), 
                    do_sample=True, max_length=42,
                    min_length=5, temperature=0.9, repetition_penalty=1.5,
                    no_repeat_ngram_size=2)[0]
                # print(f"{idx}번 인코딩 후 generate : {output_sequence}")
        
            # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
            generated_sequence = output_sequence.tolist()
            # print(f"{idx}번 인코딩 리스트 : {generated_sequence} \n")

            # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
            if tokenizer.pad_token_id in generated_sequence:
                generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
            
            generated_sequence = torch.tensor(generated_sequence) 
            # print(f"{idx}번 인코딩 리스트 패딩 제거 후 다시 텐서 : {generated_sequence} \n")

            # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
            # print(generated_sequence)
            if idx != 0:
                # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거
                generated_sequence = generated_sequence[len_sequence:]

            len_sequence = len(generated_sequence)
            # print("len_seq", len_sequence)

            # 음절 그대로 뱉으면 다시 해와, 아니면 while문 탈출
            if len_sequence > 1:
                break

        # 결과물 리스트에 담기
        res_l.append(generated_sequence)

        # print("res_l :", res_l)

    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        decode_res = tokenizer.decode(res, clean_up_tokenization_spaces=True)
        print(f"{letter} :", decode_res)

## 삼행시 함수 2

### 발라드 단어 사전 만들기

In [ ]:
root_path = "../data/data_ballad"

# 학습 데이터
all = pd.read_parquet(f"{root_path}/ballad_all.gzip")
all.shape

(54743, 6)

In [ ]:
# 가사에 개행문자 없는 데이터 제거
all = all[all["가사"].str.contains("\n")]
# 그외 전처리
all["가사"] = all["가사"].map(lambda x : re.sub("[^가-힣\n ]", "", x).strip()) # 한글 자음, 한글, 숫자, 개행문자만 남기고 제거
all["가사"] = all["가사"].map(lambda x : re.sub("\s{2,}", " ", x)) # 공백 2회 이상 제거
all = all[all["가사"].map(lambda x : len(x) > 10)] # 전처리 후 빈 행이나 10자 이상이 안되는 데이터 제거
all = all.reset_index(drop=True) # 인덱스 초기화

In [ ]:
sentence_lst = []
lyrics = all["가사"].str.split("\n")
for sentence in lyrics:
    for word in sentence:
        sentence_lst.extend(word.split())

In [ ]:
sentence = pd.Series(sentence_lst)
sentence.shape

(6638809,)

In [ ]:
sentence = sentence.drop_duplicates()
sentence.shape

(205504,)

In [ ]:
sentence = sentence[sentence.str.len() > 1]
sentence.shape

(204599,)

In [ ]:
sentence = sentence.sort_values()

In [ ]:
words = sentence.reset_index(drop=True)
words.head()

0       가가
1    가가가만히
2      가거든
3     가거들랑
4      가거라
dtype: object

In [ ]:
words.to_csv(f"{root_path}/ballad_ward.txt", index=False, encoding="cp949")

In [ ]:
pd.read_csv(f"{root_path}/ballad_ward.txt", encoding="cp949")

,0
0,가가
1,가가가만히
2,가거든
3,가거들랑
4,가거라
...,...
204594,힙합은
204595,힙합을
204596,힙합의
204597,힙합이


In [ ]:
words[words.str.startswith("박")].sample(1)

83137    박동소리
dtype: object

In [ ]:
word = words[words.str.startswith("한")].sample(1).values[0]
type(word), word

(str, '한걸음이')

### 3행시
- 첫번째 음절에 맞는 word 를 찾아서 넣어주고, generate 한다.

In [ ]:
def force_poem(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []
    len_sequence = 0

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        # 발라드에 있는 단어 적용
        try:
            word = words[words.str.startswith(val)].sample(1).values[0]
        except:
            word = val
        
        # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
        # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
        link_with_pre_sentence = (" ".join(res_l)+ " " + word + " " if idx != 0 else word).strip()
        # print(link_with_pre_sentence)

        # 연결된 문장을 인코딩
        input_ids = tokenizer.encode(link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

        # 인코딩 값으로 문장 생성
        output_sequence = model.generate(
            input_ids=input_ids.to(device), 
            do_sample=True,
            max_length=42,
            min_length=len_sequence + 2,
            temperature=0.9,
            repetition_penalty=1.5,
            no_repeat_ngram_size=2)

        # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
        generated_sequence = output_sequence.tolist()[0]

        # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
        # 사용할 generated_sequence 가 5보다 짧으면 강제적으로 길이를 8로 해준다... 
        if tokenizer.pad_token_id in generated_sequence:
            check_index = generated_sequence.index(tokenizer.pad_token_id)
            check_index = check_index if check_index-len_sequence > 3 else len_sequence + 8
            generated_sequence = generated_sequence[:check_index]

        
        # 인코딩된 word 를 기준으로 slicing 해준다. 
        word_encode = tokenizer.encode(word, add_special_tokens=False, return_tensors="pt").tolist()[0][0]
        split_index = len(generated_sequence) - 1 - generated_sequence[::-1].index(word_encode)
        
        generated_sequence = generated_sequence[split_index:]
        
        # print(tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True))
        # 다음 음절을 위해 길이 갱신
        len_sequence += len([elem for elem in generated_sequence if elem not in(tokenizer.all_special_ids)])        
        # 결과물 디코딩
        decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)


    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        print(f"{letter} :", res)